## Production

Se pueden añadir parámetros
- Cubrecierre [0, 1]: Área y peso
- Cuello [0,1]: Área y peso
- Tipo de tela

In [8]:
import pandas as pd

data = pd.read_csv('./data.csv')
data.head()

,ancho_cm,largo_m,peso_g_m2,proveedor,cubre_cierre,talla_prenda,prendas_totales
0,140,48.6,130,A,0,1,91
1,140,103.2,90,A,0,1,137
2,140,80.4,160,A,0,1,110
3,120,137.8,200,A,0,1,168
4,140,101.6,200,A,0,1,141


In [9]:
print("Null values:", data.isnull().sum(), '\n')
print("Unique values:", data.nunique(), '\n')
print("Proveedors:", data.proveedor.unique(), '\n')
print("Values per proveedor:", data.proveedor.value_counts(), '\n')
print("Data types:", data.dtypes)

Null values: ancho_cm           0
largo_m            0
peso_g_m2          0
proveedor          0
cubre_cierre       0
talla_prenda       0
prendas_totales    0
dtype: int64 

Unique values: ancho_cm              4
largo_m            1300
peso_g_m2             8
proveedor             3
cubre_cierre          2
talla_prenda          3
prendas_totales     153
dtype: int64 

Proveedors: ['A' 'B' 'C'] 

Values per proveedor: proveedor
A    3000
B    3000
C    3000
Name: count, dtype: int64 

Data types: ancho_cm             int64
largo_m            float64
peso_g_m2            int64
proveedor           object
cubre_cierre         int64
talla_prenda         int64
prendas_totales      int64
dtype: object


## Artificial Neural Network

In [20]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [17]:
data = pd.read_csv('./data.csv')

ohe = OneHotEncoder(sparse_output = False)
providers_ohe = ohe.fit_transform(data[['proveedor']])
providers_dataframe = pd.DataFrame(providers_ohe, columns = ohe.get_feature_names_out(['proveedor']))

data = pd.concat([data.drop(columns = ['proveedor']), providers_dataframe], axis = 1)
data.head()

,ancho_cm,largo_m,peso_g_m2,cubre_cierre,talla_prenda,prendas_totales,proveedor_A,proveedor_B,proveedor_C
0,140,48.6,130,0,1,91,1.0,0.0,0.0
1,140,103.2,90,0,1,137,1.0,0.0,0.0
2,140,80.4,160,0,1,110,1.0,0.0,0.0
3,120,137.8,200,0,1,168,1.0,0.0,0.0
4,140,101.6,200,0,1,141,1.0,0.0,0.0


In [18]:
X = data.drop(columns = ['prendas_totales'])
y = data['prendas_totales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [19]:
def save_objects(filenames, object):
    if len(filenames) != len(object):
        raise Exception("Filenames and objects must be the same length")
        
    for index in range(len(filenames)):
        with open(filenames[index], 'wb') as f:
            pickle.dump(object[index], f)

save_objects(['ohe.pkl', 'scaler.pkl'], [ohe, scaler])

## Implementation

In [39]:
from datetime import datetime

import tensorflow as tf
import keras
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, InputLayer

In [38]:
model = Sequential([
    InputLayer(batch_input_shape = (None, 8), name = "input_layer"),
    Dense(64, activation = "relu", name = "dense"),
    Dense(32, activation = "relu", name = "dense_1"),
    Dense(16, activation = "relu", name = "dense_2"),
    Dense(1, activation = "linear", name = "dense_3")
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,201 (12.50 KB)

 Trainable params: 3,201 (12.50 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
model.compile(
    optimizer = keras.optimizers.Adam(
        learning_rate = 0.001,
        beta_1 = 0.9,
        beta_2 = 0.999,
        epsilon = 1e-7
    ),
    loss = "mean_absolute_error",
    metrics = ["mae"]
)

early_stopping = EarlyStopping(
    monitor = 'val_loss',
    patience = 5,
    restore_best_weights = True
)

model.fit(
    X_train,
    y_train,
    epochs = 100,
    validation_split = 0.2,
    callbacks = [early_stopping]
)

model.load_weights("model.h5")
print(f"Model saved at {datetime.now().strftime('%H:%M:%S')}")

Epoch 1/100
180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 4.7762 - mae: 4.7762 - val_loss: 5.1130 - val_mae: 5.1130
Epoch 2/100
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.8001 - mae: 4.8001 - val_loss: 4.8689 - val_mae: 4.8689
Epoch 3/100
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.7952 - mae: 4.7952 - val_loss: 4.8548 - val_mae: 4.8548
Epoch 4/100
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.8068 - mae: 4.8068 - val_loss: 4.9268 - val_mae: 4.9268
Epoch 5/100
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.7930 - mae: 4.7930 - val_loss: 4.8840 - val_mae: 4.8840
Epoch 6/100
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.8160 - mae: 4.8160 - val_loss: 4.8240 - val_mae: 4.8240
Epoch 7/100
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.8015 - mae: 4.8015 - val_loss: 5.1426 - val_mae: 5.1426
Epoch 8/100
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.7971 - mae: 4.7971 - val_loss: 4.9877 - val_mae: 4.9877
Epoch 9/100
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

## Evaluation

In [43]:
model = tf.keras.models.load_model('model.h5')
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test loss: {test_loss}, Test MAE: {test_mae}")

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.8651 - mean_absolute_error: 4.8651 
Test loss: 4.865091323852539, Test MAE: 4.865091323852539


In [32]:
new_observation = {
    'ancho_cm': 140,
    'largo_m': 48.6,
    'peso_g_m2': 130,
    'cubre_cierre': 0,
    'talla_prenda': 1,
    'proveedor': 'A'    
}

new_observation = pd.DataFrame([new_observation])
providers = ohe.transform(new_observation[['proveedor']])
providers = pd.DataFrame(providers, columns = ohe.get_feature_names_out(['proveedor']))
new_observation = pd.concat([new_observation.drop(columns = ['proveedor']), providers], axis = 1)

new_observation = scaler.transform(new_observation)
model.predict(new_observation)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


array([[90.09149]], dtype=float32)